In [21]:
DEBUG = True

In [45]:
import numpy as np
from tqdm import tqdm

In [58]:
M = np.random.randint(100, 1000)
K = np.random.randint(10, 100)

u = np.random.randint(0,2, size=M)
k = np.random.randint(0,2, size=K)

if (DEBUG): 
    print(M)
    print(K)
    print(u)
    print(k)

770
64
[0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1 1 0 1
 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 1 0 1 0
 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0
 1 0 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 1 1 0
 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1
 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1 1 1 1
 1 1 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0
 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 0 1 0
 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1
 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1
 0 1 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0
 1 1 0 1 0 1 0 1 1

In [37]:
def computeTag(u, k):
    dec_u = sum(int(digit) for digit in str(int("".join(map(str, u)), 2)))
    dec_k = sum(int(digit) for digit in str(int("".join(map(str, k)), 2)))

    if DEBUG:
        print("Computed k sum: ",dec_k)

    dec_tag = dec_u*dec_k
    tag = np.array([int(digit) for digit in bin(dec_tag)[2:]])
    return tag

In [30]:
def verification (u, k, t):
    return np.array_equal(computeTag(u, k), t)


if(DEBUG):
    t = computeTag(u,k)
    print(t)
    #u = np.random.randint(0,2, size=M)
    print(verification(u,k,t))


[1 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0]
True


***TASK 2***

In [41]:
def computeAttackTag(u, dec_k):
    dec_u = sum(int(digit) for digit in str(int("".join(map(str, u)), 2)))
    dec_tag = dec_u*dec_k
    tagAttack = np.array([int(digit) for digit in bin(dec_tag)[2:]])
    return tagAttack 

In [47]:
def sub_attack(u,t):
    dec_u = sum(int(digit) for digit in str(int("".join(map(str, u)), 2)))
    dec_tag = int("".join(map(str, t)), 2)
    dec_key = dec_tag//dec_u

    if DEBUG:
        print("Attacker computed k sum: ", dec_key)

    u1 = np.random.randint(0,2, size=M)
    tag1 = computeAttackTag(u1, dec_key)
    return u1, tag1

if (DEBUG):
    success = 0
    for i in tqdm(range(10000)):
        u = np.random.randint(0,2, size=M)
        k = np.random.randint(0,2, size=K)
        ogTag = computeTag(u, k)
        u1, tag1 = sub_attack(u, ogTag)
        if(verification(u1, k, tag1)):
            success += 1
    prob = success/10000
    print(prob)

  1%|▏         | 148/10000 [00:00<00:15, 654.91it/s]

Computed k sum:  94
Attacker computed k sum:  94
Computed k sum:  94
Computed k sum:  109
Attacker computed k sum:  109
Computed k sum:  109
Computed k sum:  125
Attacker computed k sum:  125
Computed k sum:  125
Computed k sum:  106
Attacker computed k sum:  106
Computed k sum:  106
Computed k sum:  110
Attacker computed k sum:  110
Computed k sum:  110
Computed k sum:  98
Attacker computed k sum:  98
Computed k sum:  98
Computed k sum:  100
Attacker computed k sum:  100
Computed k sum:  100
Computed k sum:  122
Attacker computed k sum:  122
Computed k sum:  122
Computed k sum:  134
Attacker computed k sum:  134
Computed k sum:  134
Computed k sum:  134
Attacker computed k sum:  134
Computed k sum:  134
Computed k sum:  113
Attacker computed k sum:  113
Computed k sum:  113
Computed k sum:  109
Attacker computed k sum:  109
Computed k sum:  109
Computed k sum:  138
Attacker computed k sum:  138
Computed k sum:  138
Computed k sum:  116
Attacker computed k sum:  116
Computed k sum:  11

  4%|▎         | 364/10000 [00:00<00:13, 698.84it/s]

Computed k sum:  128
Computed k sum:  118
Attacker computed k sum:  118
Computed k sum:  118
Computed k sum:  117
Attacker computed k sum:  117
Computed k sum:  117
Computed k sum:  121
Attacker computed k sum:  121
Computed k sum:  121
Computed k sum:  118
Attacker computed k sum:  118
Computed k sum:  118
Computed k sum:  102
Attacker computed k sum:  102
Computed k sum:  102
Computed k sum:  158
Attacker computed k sum:  158
Computed k sum:  158
Computed k sum:  128
Attacker computed k sum:  128
Computed k sum:  128
Computed k sum:  126
Attacker computed k sum:  126
Computed k sum:  126
Computed k sum:  118
Attacker computed k sum:  118
Computed k sum:  118
Computed k sum:  105
Attacker computed k sum:  105
Computed k sum:  105
Computed k sum:  112
Attacker computed k sum:  112
Computed k sum:  112
Computed k sum:  135
Attacker computed k sum:  135
Computed k sum:  135
Computed k sum:  105
Attacker computed k sum:  105
Computed k sum:  105
Computed k sum:  111
Attacker computed k su

  6%|▌         | 615/10000 [00:00<00:11, 805.63it/s]

...

Computed k sum:  126
Attacker computed k sum:  126
Computed k sum:  126
Computed k sum:  141
Attacker computed k sum:  141
Computed k sum:  141
Computed k sum:  135
Attacker computed k sum:  135
Computed k sum:  135
Computed k sum:  111
Attacker computed k sum:  111
Computed k sum:  111
Computed k sum:  126
Attacker computed k sum:  126
Computed k sum:  126
Computed k sum:  115
Attacker computed k sum:  115
Computed k sum:  115
Computed k sum:  104
Attacker computed k sum:  104
Computed k sum:  104
Computed k sum:  130
Attacker computed k sum:  130
Computed k sum:  130
Computed k sum:  114
Attacker computed k sum:  114
Computed k sum:  114
Computed k sum:  117
Attacker computed k sum:  117
Computed k sum:  117
Computed k sum:  154
Attacker computed k sum:  154
Computed k sum:  154
Computed k sum:  108
Attacker computed k sum:  108
Computed k sum:  108
Computed k sum:  110
Attacker computed k sum:  110
Computed k sum:  110
Computed k sum:  118
Attacker computed k sum:  118
Computed k su

 98%|█████████▊| 9844/10000 [00:12<00:00, 889.01it/s]

Computed k sum:  128
Computed k sum:  128
Attacker computed k sum:  128
Computed k sum:  128
Computed k sum:  108
Attacker computed k sum:  108
Computed k sum:  108
Computed k sum:  93
Attacker computed k sum:  93
Computed k sum:  93
Computed k sum:  124
Attacker computed k sum:  124
Computed k sum:  124
Computed k sum:  110
Attacker computed k sum:  110
Computed k sum:  110
Computed k sum:  135
Attacker computed k sum:  135
Computed k sum:  135
Computed k sum:  115
Attacker computed k sum:  115
Computed k sum:  115
Computed k sum:  135
Attacker computed k sum:  135
Computed k sum:  135
Computed k sum:  96
Attacker computed k sum:  96
Computed k sum:  96
Computed k sum:  99
Attacker computed k sum:  99
Computed k sum:  99
Computed k sum:  108
Attacker computed k sum:  108
Computed k sum:  108
Computed k sum:  108
Attacker computed k sum:  108
Computed k sum:  108
Computed k sum:  132
Attacker computed k sum:  132
Computed k sum:  132
Computed k sum:  99
Attacker computed k sum:  99
Com

100%|██████████| 10000/10000 [00:12<00:00, 790.09it/s]

Computed k sum:  136
Attacker computed k sum:  136
Computed k sum:  136
Computed k sum:  96
Attacker computed k sum:  96
Computed k sum:  96
Computed k sum:  152
Attacker computed k sum:  152
Computed k sum:  152
Computed k sum:  106
Attacker computed k sum:  106
Computed k sum:  106
Computed k sum:  124
Attacker computed k sum:  124
Computed k sum:  124
Computed k sum:  128
Attacker computed k sum:  128
Computed k sum:  128
Computed k sum:  105
Attacker computed k sum:  105
Computed k sum:  105
Computed k sum:  114
Attacker computed k sum:  114
Computed k sum:  114
Computed k sum:  134
Attacker computed k sum:  134
Computed k sum:  134
Computed k sum:  117
Attacker computed k sum:  117
Computed k sum:  117
Computed k sum:  100
Attacker computed k sum:  100
Computed k sum:  100
Computed k sum:  146
Attacker computed k sum:  146
Computed k sum:  146
Computed k sum:  120
Attacker computed k sum:  120
Computed k sum:  120
Computed k sum:  102
Attacker computed k sum:  102
Computed k sum: 

***TASK 3***

In [ ]:
import math

#K = 10

def kSumProbability():
    dict = {}

    for k in range(int(math.pow(2, K))):
        s = sum(int(digit) for digit in str(k))

        if dict.get(s) == None:
            dict[s] = 1
        else:
            dict[s] += 1

    #print(dict)
    print(max(dict, key=dict.get))

kSumProbability()


porca madonna roto vuole il grafico